In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings
from data_models.flight_data_models import FlightSearchResults, BestFlight, AirportInfo, Flight, Layover, OtherFlight, PriceInsights, AirportLocation, Airports

from travel_agents import question_rewrite_agent, flight_agent
from functions import flight_search
from pprint import pprint

END OF THE SCRIPT


In [2]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = pekin

main_question = f"""I want to travel from {start_point} to {dest_point}. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
In addition to that, I like eating sea food. I guess {dest_point} is very rich city interms of sea restaurants""".strip()

question_rewrite_agent_output = Runner.run_sync(question_rewrite_agent,main_question)

In [3]:
question_rewrite_agent_output.final_output

FinalModel(flight_reqs=FlightOutput(departure_id='ESB', arrival_id='PEK', outbound_data='2025-05-05', return_data='2025-05-11'), yelp_reqs=YelpOutput(search_term='sea food', location='Peking'), hotel_reqs=HotelOutput(query='Peking', check_in_date='2025-05-05', check_out_date='2025-05-11', gl='non-smoking'))

In [4]:
question_rewrite_agent_output.final_output

FinalModel(flight_reqs=FlightOutput(departure_id='ESB', arrival_id='PEK', outbound_data='2025-05-05', return_data='2025-05-11'), yelp_reqs=YelpOutput(search_term='sea food', location='Peking'), hotel_reqs=HotelOutput(query='Peking', check_in_date='2025-05-05', check_out_date='2025-05-11', gl='non-smoking'))

In [5]:
question_rewrite_agent_output.final_output.flight_reqs.return_data

'2025-05-11'

In [6]:
# flight_results = Runner.run_sync(flight_agent,main_question)

In [19]:
x = flight_search(
    # departure_id= "ESB",
    # arrival_id='AMS',
    departure_id=question_rewrite_agent_output.final_output.flight_reqs.departure_id, 
    arrival_id= question_rewrite_agent_output.final_output.flight_reqs.arrival_id ,
    outbound_data= question_rewrite_agent_output.final_output.flight_reqs.outbound_data, 
    return_date = question_rewrite_agent_output.final_output.flight_reqs.return_data
    )

flight_results = FlightSearchResults.parse_obj(x)  # Use parse_obj with dict


C:\Users\user\AppData\Local\Temp\ipykernel_21860\3572277655.py:10: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  flight_results = FlightSearchResults.parse_obj(x)  # Use parse_obj with dict


In [20]:
x

{'search_metadata': {'id': '67fd627ea8dd4ea9275d65af',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/12487f80c39b39af/67fd627ea8dd4ea9275d65af.json',
  'created_at': '2025-04-14 19:31:10 UTC',
  'processed_at': '2025-04-14 19:31:10 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTA1LTA1agcIARIDRVNCcgcIARIDUEVLGh4SCjIwMjUtMDUtMTFqBwgBEgNQRUtyBwgBEgNFU0JCAQFIAXABmAEB&tfu=EgIIAQ',
  'raw_html_file': 'https://serpapi.com/searches/12487f80c39b39af/67fd627ea8dd4ea9275d65af.html',
  'prettify_html_file': 'https://serpapi.com/searches/12487f80c39b39af/67fd627ea8dd4ea9275d65af.prettify',
  'total_time_taken': 0.74},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'departure_id': 'ESB',
  'arrival_id': 'PEK',
  'outbound_date': '2025-05-05',
  'return_date': '2025-05-11',
  'currency': 'USD',
  'deep_search': True},
 'best_flights': [{'flights': [{'departure_airport': {'name'

In [23]:
x.keys()

dict_keys(['search_metadata', 'search_parameters', 'best_flights', 'other_flights', 'price_insights', 'airports'])

In [24]:
x['other_flights']

[{'flights': [{'departure_airport': {'name': 'Ankara Esenboga Airport',
     'id': 'ESB',
     'time': '2025-05-05 12:00'},
    'arrival_airport': {'name': 'Istanbul Airport',
     'id': 'IST',
     'time': '2025-05-05 13:20'},
    'duration': 80,
    'airplane': 'Boeing 737',
    'airline': 'Turkish Airlines',
    'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/TK.png',
    'travel_class': 'Economy',
    'flight_number': 'TK 2139',
    'ticket_also_sold_by': ['THAI'],
    'legroom': '31 in',
    'extensions': ['Average legroom (31 in)',
     'Wi-Fi for a fee',
     'In-seat USB outlet',
     'On-demand video',
     'Carbon emissions estimate: 67 kg']},
   {'departure_airport': {'name': 'Istanbul Airport',
     'id': 'IST',
     'time': '2025-05-05 16:30'},
    'arrival_airport': {'name': 'Suvarnabhumi Airport',
     'id': 'BKK',
     'time': '2025-05-06 05:40'},
    'duration': 550,
    'airplane': 'Airbus A350',
    'airline': 'THAI',
    'airline_logo': 'https://

In [27]:
for i in x['other_flights']:
    # print(i)
    y = OtherFlight(**i)
    print(f"""
✈️ Flight Summary:
Type: {y.type}
Total Duration: {y.total_duration} minutes
Price: ${y.price}
Airline Logo: {y.airline_logo or 'N/A'}
Carbon Emissions: {y.carbon_emissions.this_flight / 1000:.1f} kg (Typical: {y.carbon_emissions.typical_for_this_route / 1000:.1f} kg, Difference: {y.carbon_emissions.difference_percent}%)
""")


✈️ Flight Summary:
Type: Round trip
Total Duration: 1365 minutes
Price: $1773
Airline Logo: https://www.gstatic.com/flights/airline_logos/70px/multi.png
Carbon Emissions: 833.0 kg (Typical: 615.0 kg, Difference: 35%)


✈️ Flight Summary:
Type: Round trip
Total Duration: 2500 minutes
Price: $2229
Airline Logo: https://www.gstatic.com/flights/airline_logos/70px/multi.png
Carbon Emissions: 680.0 kg (Typical: 615.0 kg, Difference: 11%)


✈️ Flight Summary:
Type: Round trip
Total Duration: 1000 minutes
Price: $2351
Airline Logo: https://www.gstatic.com/flights/airline_logos/70px/multi.png
Carbon Emissions: 769.0 kg (Typical: 615.0 kg, Difference: 25%)


✈️ Flight Summary:
Type: Round trip
Total Duration: 950 minutes
Price: $3343
Airline Logo: https://www.gstatic.com/flights/airline_logos/70px/multi.png
Carbon Emissions: 708.0 kg (Typical: 615.0 kg, Difference: 15%)


✈️ Flight Summary:
Type: Round trip
Total Duration: 1540 minutes
Price: $None
Airline Logo: https://www.gstatic.com/flights/

In [31]:
x['other_flights']

[{'flights': [{'departure_airport': {'name': 'Ankara Esenboga Airport',
     'id': 'ESB',
     'time': '2025-05-05 12:00'},
    'arrival_airport': {'name': 'Istanbul Airport',
     'id': 'IST',
     'time': '2025-05-05 13:20'},
    'duration': 80,
    'airplane': 'Boeing 737',
    'airline': 'Turkish Airlines',
    'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/TK.png',
    'travel_class': 'Economy',
    'flight_number': 'TK 2139',
    'ticket_also_sold_by': ['THAI'],
    'legroom': '31 in',
    'extensions': ['Average legroom (31 in)',
     'Wi-Fi for a fee',
     'In-seat USB outlet',
     'On-demand video',
     'Carbon emissions estimate: 67 kg']},
   {'departure_airport': {'name': 'Istanbul Airport',
     'id': 'IST',
     'time': '2025-05-05 16:30'},
    'arrival_airport': {'name': 'Suvarnabhumi Airport',
     'id': 'BKK',
     'time': '2025-05-06 05:40'},
    'duration': 550,
    'airplane': 'Airbus A350',
    'airline': 'THAI',
    'airline_logo': 'https://

OtherFlight(flights=[Flight(departure_airport=AirportInfo(name='Ankara Esenboga Airport', id='ESB', time='2025-05-05 12:00'), arrival_airport=AirportInfo(name='Istanbul Airport', id='IST', time='2025-05-05 13:20'), duration=80, airplane='Boeing 737', airline='Turkish Airlines', airline_logo='https://www.gstatic.com/flights/airline_logos/70px/TK.png', travel_class='Economy', flight_number='TK 2139', legroom='31 in', extensions=['Average legroom (31 in)', 'Wi-Fi for a fee', 'In-seat USB outlet', 'On-demand video', 'Carbon emissions estimate: 67 kg'], overnight=None, ticket_also_sold_by=['THAI'], plane_and_crew_by=None), Flight(departure_airport=AirportInfo(name='Istanbul Airport', id='IST', time='2025-05-05 16:30'), arrival_airport=AirportInfo(name='Suvarnabhumi Airport', id='BKK', time='2025-05-06 05:40'), duration=550, airplane='Airbus A350', airline='THAI', airline_logo='https://www.gstatic.com/flights/airline_logos/70px/TG.png', travel_class='Economy', flight_number='TG 901', legroom

In [39]:
flight = OtherFlight(**x['other_flights'][1])

# Loop through flights and embed layovers
for i, f in enumerate(flight.flights):
    text_summary += f"""
--- Segment {i + 1} ---
Airline: {f.airline or 'N/A'} ({f.flight_number})
From: {f.departure_airport.name} ({f.departure_airport.id}) at {f.departure_airport.time}
To: {f.arrival_airport.name} ({f.arrival_airport.id}) at {f.arrival_airport.time}
Duration: {f.duration} minutes
Airplane: {f.airplane}
Class: {f.travel_class}
Legroom: {f.legroom or 'N/A'}
Extensions: {', '.join(f.extensions or [])}
"""

    # Only insert a layover if one exists after this flight
    if flight.layovers and i < len(flight.layovers):
        layover = flight.layovers[i]
        text_summary += f"Layover at {layover.name} ({layover.id}) - {layover.duration} minutes\n"

text_summary += f"\nDeparture Token: {flight.departure_token}"

print(text_summary)



✈️ Flight Summary:
Type: Round trip
Total Duration: 1365 minutes
Price: $1773
Airline Logo: https://www.gstatic.com/flights/airline_logos/70px/multi.png
Carbon Emissions: 833.0 kg (Typical: 615.0 kg, Difference: 35%)

--- Segment 1 ---
Airline: Turkish Airlines (TK 2139)
From: Ankara Esenboga Airport (ESB) at 2025-05-05 12:00
To: Istanbul Airport (IST) at 2025-05-05 13:20
Duration: 80 minutes
Airplane: Boeing 737
Class: Economy
Legroom: 31 in
Extensions: Average legroom (31 in), Wi-Fi for a fee, In-seat USB outlet, On-demand video, Carbon emissions estimate: 67 kg
Layover at Istanbul Airport (IST) - 190 minutes

--- Segment 2 ---
Airline: THAI (TG 901)
From: Istanbul Airport (IST) at 2025-05-05 16:30
To: Suvarnabhumi Airport (BKK) at 2025-05-06 05:40
Duration: 550 minutes
Airplane: Airbus A350
Class: Economy
Legroom: 31 in
Extensions: Average legroom (31 in), In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 511 kg
Layover at Suvarnabhumi Airport (BKK) - 260 min

In [ ]:
elma

In [ ]:
x

In [ ]:

flight_info = ""
c = 0
for j in flight_results.best_flights:
  for i, flight in enumerate(j.flights, start=1):
      c += 1
      x= f"""
  Flight {c}:
    Departure: {flight.departure_airport.name} ({flight.departure_airport.id}) at {flight.departure_airport.time}
    Arrival: {flight.arrival_airport.name} ({flight.arrival_airport.id}) at {flight.arrival_airport.time}
    Duration: {flight.duration} minutes
    Airline: {flight.airline}
    Airplane: {flight.airplane}
    Class: {flight.travel_class}
    Flight Number: {flight.flight_number}
    Legroom: {flight.legroom}
    Extensions: {', '.join(flight.extensions)}
  """
      flight_info += x


for j in flight_results.other_flights:
  for i, flight in enumerate(j.flights, start=1):
      c += 1
      x= f"""
  Flight {c}:
    Departure: {flight.departure_airport.name} ({flight.departure_airport.id}) at {flight.departure_airport.time}
    Arrival: {flight.arrival_airport.name} ({flight.arrival_airport.id}) at {flight.arrival_airport.time}
    Duration: {flight.duration} minutes
    Airline: {flight.airline}
    Airplane: {flight.airplane}
    Class: {flight.travel_class}
    Flight Number: {flight.flight_number}
    Legroom: {flight.legroom}
    Extensions: {', '.join(flight.extensions)}
  """
      flight_info += x


In [ ]:
print(flight_info)

In [ ]:
f_agent = Agent(name="Flight Assistant agent",
              instructions=f"You are helpful flight asisstants try to answer the questions according to: \n\n{flight_info}. You are not allowed to recommend or answer any other thing than flights",
            #   model="gpt-4o-mini",
              model = 'gpt-4o',
              model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*8),
              )

In [ ]:
result = Runner.run_sync(f_agent, main_question+" bring me cheapest flight, also tell me how much does that cost?")
print(result.final_output)

In [ ]:
result = Runner.run_sync(f_agent, "list back all the flights")
print(result.final_output)